In [66]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key= os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings =HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [67]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

In [68]:
import bs4
loader = WebBaseLoader(
    web_paths=("https://docs.python.org/3.13/whatsnew/3.13.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("document","related","sphinxsidebar")
        )
    ),
)

docs=loader.load()

In [69]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorestore = Chroma.from_documents(documents=splits,embedding=embeddings)
retriever = vectorestore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f04ea5eacf0>)

In [70]:
# Prompt Template

system_prompt = (
    "You are an assistant for question-answering task"
    "Use the following pieces of retrived context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [71]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever, question_answer_chain)

In [72]:
response = rag_chain.invoke({"input":"What are the new feature?"})
response

{'input': 'What are the new feature?',
 'context': [Document(metadata={'source': 'https://docs.python.org/3.13/whatsnew/3.13.html'}, page_content='Navigation\n\n\nindex\n\nmodules |\n\nnext |\n\nprevious |\n\nPython »\n\n\n\n\n\n\n\n3.13.0b4 Documentation »\n    \nWhat’s New in Python »\nWhat’s New In Python 3.13\n\n\n\n\n\n\n\n                     |\n                \n\n\n    Theme\n    \nAuto\nLight\nDark\n\n |\n\n\n\n\n\n\nWhat’s New In Python 3.13¶\n\nEditor:\nThomas Wouters\n\n\nThis article explains the new features in Python 3.13, compared to 3.12.\nFor full details, see the changelog.\n\nSee also\nPEP 719 – Python 3.13 Release Schedule\n\n\nNote\nPrerelease users should be aware that this document is currently in draft\nform. It will be updated substantially as Python 3.13 moves towards release,\nso it’s worth checking back even after reading earlier versions.'),
  Document(metadata={'source': 'https://docs.python.org/3.13/whatsnew/3.13.html'}, page_content='Navigation\n\n\nind

In [73]:
rag_chain.invoke({"input":"How it handles garbege collection?"})

{'input': 'How it handles garbege collection?',
 'context': [Document(metadata={'source': 'https://docs.python.org/3.13/whatsnew/3.13.html'}, page_content='gc¶\n\nThe cyclic garbage collector is now incremental, which changes the meanings\nof the results of gc.get_threshold() and gc.set_threshold() as\nwell as gc.get_count() and gc.get_stats().\n\ngc.get_threshold() returns a three-item tuple for backwards compatibility.\nThe first value is the threshold for young collections, as before; the second\nvalue determines the rate at which the old collection is scanned (the\ndefault is 10, and higher values mean that the old collection is scanned more slowly).\nThe third value is meaningless and is always zero.\ngc.set_threshold() ignores any items after the second.\ngc.get_count() and gc.get_stats()\nreturn the same format of results as before.\nThe only difference is that instead of the results referring to\nthe young, aging and old generations, the results refer to the\nyoung generation a

In [74]:
# Adding Chat History
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder


contextualize_q_system_prompt = (
    """
    Given a chat history and the latest user question
    which might reference context in the chat history,
    formulate a standalone question which can be understood
    without the chat history. Do NOT answer the question,
    just reformulation it if needed and otherwise return it as is.
    """
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [75]:
history_aware_retriver=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriver

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f04ea5eacf0>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n    Given a chat history and the latest user question\n    which might reference context in the chat history,\n    formulate a standalone question which can be understood\n    without the chat history. Do NOT answer the question,

In [76]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [77]:
# question_answer_chain= create_stuff_documents_chain(llm,qa_prompt)
question_answer_chain= create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriver,question_answer_chain)

In [79]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question = "What is the garbage collection"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response["answer"])
    ]
)

question2="Tell me more about it?"
response2 = rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

The garbage collector is a mechanism that periodically reclaims memory occupied by objects that are no longer needed or referenced in a program.


In [80]:
chat_history

[HumanMessage(content='What is the garbage collection'),
 AIMessage(content="I don't know.")]